# Predict parent neuron
Using Jasper's idea to predict parent neurons for nuclei, Thank you so much, Jasper.

In [1]:
# libraries 1
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sys
import os
import pandas as pd
import csv
from tqdm import tqdm
from scipy import ndimage

from cloudvolume import CloudVolume, view, Bbox
import cc3d
from tifffile.tifffile import imwrite

In [2]:
# libraries 2
# %pwd
sys.path.append(os.path.abspath("../segmentation"))
# to import rootID_lookup and authentication_utils like below

import rootID_lookup as IDlook
import authentication_utils as auth

In [3]:
np.random.seed(123)

In [4]:
# cv setting
seg = CloudVolume(auth.get_cv_path('FANC_production_segmentation')['url'], use_https=True, agglomerate=False, cache=True, progress=False)

In [5]:
# read csv

In [6]:
# cord = (50816, 85456,  1880) #mip0
id = (648518346490989503)

In [7]:
# cord[0]

In [ ]:
# change coordination from mip0 to mip2
""" coord_mip2 = cord
coord_mip2[0]  = (cord[0] /(2**2))
coord_mip2[1]  = (cord[1] /(2**2))
coord_mip2 = coord_mip2.astype('int64')
coord_mip2 """

In [8]:
cord_mip2 = np.array([12704, 21364,  1880]) #mip2


In [9]:
type(id)
size_xy = 160 # 160/(2**2)??
# 128x128x160 is small

In [10]:
seg_nuc = seg.download_point(pt=cord_mip2, segids=id, size=[size_xy, size_xy, 160], coord_resolution=[17.2, 17.2, 45.0])
# lowest resolution of seg is [17.2, 17.2, 45.0]

In [11]:
# view(seg_nuc, segmentation=True)

In [12]:
vol_temp = seg_nuc[:,:,:]
vol_temp[vol_temp>0] = 1 # change segID assigned to each cell body into 1
vol = np.squeeze(vol_temp)

In [13]:
# save images
# volume = vol.swapaxes(0, 2).astype('float32')
# imwrite('../Output/vol.tif', volume, imagej=True)

In [29]:
# fill the empty space with one 
filled = ndimage.binary_fill_holes(vol).astype(int)
filled2 = filled.astype('float32')
filled2[filled2>0] = 1

In [30]:
volume = filled2.swapaxes(0, 2)
imwrite('../Output/vol_filled2.tif', volume, imagej=True)

In [17]:
def vol_shift(input): # Although np.roll is fast, this is very slow since this overuse RAM
    # x plane
    x_p = np.roll(input, 1, axis=0)
    x_p[0,:,:] = 0
    x_n = np.roll(input, -1, axis=0)
    x_n[-1,:,:] = 0
    # y plane
    y_p = np.roll(input, 1, axis=1)
    y_p[:,0,:] = 0
    y_n = np.roll(input, -1, axis=1)
    y_n[:,-1,:] = 0
    # z plane
    z_p = np.roll(input, 1, axis=2)
    z_p[:,:,0] = 0
    z_n = np.roll(input, -1, axis=2)
    z_n[:,:,-1] = 0

    sum = x_p + x_n + y_p + y_n + z_p + z_n
    result = sum - input*6

    return result

In [18]:
shifted = vol_shift(filled)
shifted = shifted.astype('float32')
shifted[shifted>0] = 1
volume = shifted.swapaxes(0, 2)
imwrite('../Output/vol_shifted.tif', volume, imagej=True)

In [ ]:
location_one = np.argwhere(shifted == 1)
len(location_one) #143518 
# I need to decrease the number...

In [ ]:
if len(location_one):
    origin = seg_nuc.bounds.minpt # 3072,5248,1792
    parent_coordinates_mip2 = np.add(np.array(location_one), origin)
    parent_coordinates = parent_coordinates_mip2
    parent_coordinates[:,0]  = (parent_coordinates_mip2[:,0] * 2**2)
    parent_coordinates[:,1]  = (parent_coordinates_mip2[:,1] * 2**2)
    parent_coordinates = parent_coordinates.astype('int64')

    #randomly select 10
    index = np.random.choice(parent_coordinates.shape[0], size=10, replace=False)
    location_random = parent_coordinates[index]

    # Lets get IDs using cell_body_coordinates
    parent_IDs = IDlook.segIDs_from_pts_cv(pts=location_random, cv=seg) #mip0

    # save
    # type(cell_body_coordinates.shape)
    # cord_pd = pd.DataFrame(cell_body_coordinates, columns=["x", "y", "z"])
    # temp = cord_pd
    # temp['segIDs'] = cell_body_IDs
    # output.append(temp)
else:
    pass

In [ ]:
parent_IDs 

In [ ]:
index = np.random.choice(parent_coordinates.shape[0], size=10, replace=False)
location_random = parent_coordinates[index]


In [ ]:
parent_coordinates.shape

In [ ]:
seg_nuc.cache.flush()

In [ ]:
# nuclei_cv.mip_volume_size(0)
# Vec(86016,225776,4390, dtype=int64)
# entire dataset
# [ 83968 223232   4390]
# why different

In [ ]:
# fill gap inside cel bodies with one?
#  maybe this gap is detected with our get_info

# read csv 
# output csv